In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Read data

In [20]:
with open("heats.txt", "r") as f:
        columns = f.readline()
        data = f.readlines()

In [21]:
columns = columns.strip("\n").split("\t")
print("Data has {} columns: {}".format(len(columns), columns))

Data has 10 columns: ['"sbp"', '"tobacco"', '"ldl"', '"adiposity"', '"famhist"', '"typea"', '"obesity"', '"alcohol"', '"age"', '"chd"']


In [22]:
def clean(s):
    s = s.strip("\n").split("\t")
    return s
data = list(map(clean, data))
data = list(filter(lambda d:len(d)!=1, data))
print("There are totally {} samples.".format(len(data)))

There are totally 462 samples.


In [23]:
# Change famhist label to int
i = 0
for d in data:
    d[4] = int(d[4] == '"Present"')
    d = [float(e) for e in d]

In [24]:
data = np.array(data)
train_feature = data[:400, 0:9]
train_label = data[:400, 9]
test_feature = data[400:, 0:9]
test_label = data[400:, 9]
print("Data has shape: {}".format(data.shape))
print("Train feature has shape: {}".format(train_feature.shape))
print("Train label has shape: {}".format(train_label.shape))
print("Test feature has shape: {}".format(test_feature.shape))
print("Test label has shape: {}".format(test_label.shape))

Data has shape: (462, 10)
Train feature has shape: (400, 9)
Train label has shape: (400,)
Test feature has shape: (62, 9)
Test label has shape: (62,)


In [25]:
def next_batch(data, label, batch_size):
    data = np.hstack([train_feature, train_label.reshape(400,1)])
    np.random.shuffle(data)
    feature = data[:batch_size, 0:9]
    label = data[:batch_size, 9]
    return feature, label
next_feature, next_label = next_batch(train_feature, train_label, 256)
print(next_feature.shape)
print(next_label.shape)

(256, 9)
(256,)


 ## Define parameter      

In [26]:
batch_size = 128
learning_rate = 0.001
tf.set_random_seed(0)

## Define placeholder

In [27]:
X = tf.placeholder(tf.float32, shape=[None, 9], name="Input")
Y = tf.placeholder(tf.float32, shape=[None], name="Label")

## Define Variables

In [28]:
w1 = tf.Variable(tf.truncated_normal([9, 1], stddev=1.0), name="Weights1")
b1 = tf.Variable(tf.zeros([]), name="Bias1")

## Define sigmoid loss

In [29]:
with tf.name_scope("Loss"):
    logits = tf.matmul(X, w1) + b1
    loss = tf.reduce_mean(tf.square(tf.sigmoid(logits) - Y))

## Define accuracy

In [30]:
with tf.name_scope("Accuracy"):
    predictions = tf.cast(tf.sigmoid(logits) > 0.5 , tf.float32)
    #accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, Y), tf.float32))
    accuracy, accuracy_op = tf.metrics.accuracy(Y, predictions)

## Define train_op

In [31]:
with tf.name_scope("Train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)

## Define summary op

In [32]:
with tf.name_scope("Summaries"):
    loss_summary = tf.summary.scalar("loss", loss)
    tf.summary.histogram("histogram_loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    summary_op = tf.summary.merge_all()

## Train

In [33]:
loss_history = []
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
writer = tf.summary.FileWriter("/tmp/graph/lg", sess.graph)
for i in range(200):
    next_feature, next_label = next_batch(train_feature, train_label, 400)
    _, np_loss, np_accuracy, summary = sess.run([train_op, loss, accuracy, summary_op],
                        feed_dict={X:next_feature, Y:next_label})
    print("Loss: {} Acc: {}".format(np_loss, np_accuracy))
    loss_history.append(np_loss)
    writer.add_summary(summary, global_step=i)

InvalidArgumentError: You must feed a value for placeholder tensor 'Input' with dtype float and shape [?,9]
	 [[Node: Input = Placeholder[dtype=DT_FLOAT, shape=[?,9], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Input', defined at:
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/wang/denv/.env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/wang/denv/.env/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 405, in start
 

In [ ]:
# test model
test_accuracy = sess.run(accuracy_op, feed_dict={X:test_feature, Y:test_label})
print("Test accuracy: {}".format(test_accuracy))
print(sess.run([w1, b1]))
print(sess.run([logits,predictions], feed_dict={X:test_feature, Y:test_label}))

In [ ]:
plt.plot(loss_history)
plt.title('Loss history')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

!tensorboard --logdir="/tmp/graph/lg"